In [12]:
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import tensorflow as tf


def init_normal(shape=[0, 0.05], seed=None):
    mean, stddev = shape
    return keras.initializers.RandomNormal(
        mean=mean, stddev=stddev, seed=seed)


def normalize(tensor):
    K.l2_normalize(tensor)
    return tensor


def get_model(num_users, num_items, num_tasks, e_dim=16, f_dim=8, reg=0):
    """
    This function is used to get the Att-Mul-MF model described
    in the paper.
    Args:
        :param num_users: number of users in the dataset
        :param num_items: number of items in the dataset
        :param num_tasks: number of tasks (item genres)
        :param e_dim: the embedding dimension
        :param f_dim: the preference feature space dimension
        :param reg: regularization coefficient
    """
    # Input variables
    user_input = keras.layers.Input(shape=(1,), dtype='int32',
                                    name='user_input')
    item_input = keras.layers.Input(shape=(1,), dtype='int32',
                                    name='item_input')

    gmf_user_embedding = keras.layers.Embedding(
        input_dim=num_users, output_dim=int(e_dim),
        name='gmf_user_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    gmf_item_embedding = keras.layers.Embedding(
        input_dim=num_items, output_dim=int(e_dim),
        name='gmf_item_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    user_embedding = keras.layers.Embedding(
        input_dim=num_users, output_dim=int(f_dim),
        name='user_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    item_embedding = keras.layers.Embedding(
        input_dim=num_items, output_dim=int(e_dim),
        name='item_embedding',
        embeddings_initializer=init_normal(),
        embeddings_regularizer=keras.regularizers.l2(reg),
        input_length=1)

    # Flatten the output tensor
    gmf_user_latent = keras.layers.Flatten()(gmf_user_embedding(user_input))
    gmf_item_latent = keras.layers.Flatten()(gmf_item_embedding(item_input))
    user_latent = keras.layers.Flatten()(user_embedding(user_input))
    item_latent = keras.layers.Flatten()(item_embedding(item_input))

    # GMF part
    gmf_vector = keras.layers.Concatenate()([gmf_user_latent, gmf_item_latent])

    # user vector feature extraction, split at the last layer
    user_vector = keras.layers.Dense(
        units=num_tasks*f_dim,
        activation='relu',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=keras.regularizers.l2(reg),
        name='user_vector')(user_latent)
    user_vector = keras.layers.Reshape(
        (num_tasks, f_dim), name='multitask_user_vector')(user_vector)

    # item vector feature extraction, split at the last layer
    item_vector = keras.layers.Dense(
        units=num_tasks*f_dim,
        activation='relu',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=keras.regularizers.l2(reg),
        name='item_vector')(item_latent)
    item_vector = keras.layers.Reshape(
        (num_tasks, f_dim), name='multitask_item_vector')(item_vector)

    weight_vector = keras.layers.Dot(axes=-1, normalize=True)(
        [user_vector, item_vector])
    weight_vector = tf.linalg.tensor_diag(weight_vector)
    att_vector = keras.layers.Dot(axes=(-1, -2))([weight_vector, item_vector])
    att_vector = keras.layers.Flatten(name='attention_layer')(att_vector)

    #  Concatenate gmf_vector and att_vector
    pred_vector = keras.layers.Concatenate()([gmf_vector, att_vector])

    prediction = keras.layers.Dense(
        units=1, activation='sigmoid',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=keras.regularizers.l2(reg),
        name='prediction')(pred_vector)
    
    aux_vector = keras.layers.Dense(
        units=1, activation='sigmoid',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=keras.regularizers.l2(reg),
        name='aux_vector')(item_vector)
    aux_vector = keras.layers.Flatten(name='aux_output')(aux_vector)

    model = keras.models.Model(inputs=[user_input, item_input],
                               outputs=[prediction, aux_vector])

    return model


In [13]:
class Args(object):
    """Used to generate different sets of arguments"""
    def __init__(self):
        # default vaules
        self.path = 'Data/'
        self.dataset = 'ml-100k'
        self.epochs = 20
        self.batch_size = 256
        self.num_tasks = 18
        self.e_dim = 8
        self.f_dim = 8
        self.reg = 0
        self.num_neg = 4
        self.lr = 0.001
        self.loss_weights = [1, 0.1]
        self.K = 10
        # self.learner = 'adam' 
args=Args()
if args.dataset == 'ml-1m':
    num_users = 6040
    num_items = 3952  # need modification
elif args.dataset == 'ml-100k':
    num_users = 943
    num_items = 1682
else:
    raise Exception('wrong dataset size!!!')
    
model = get_model(num_users,
                  num_items,
                  num_tasks=args.num_tasks,
                  e_dim=args.e_dim,
                  f_dim=args.f_dim,
                  reg=args.reg)

ValueError: Dimension 2 in both shapes must be equal, but are 18 and 8. Shapes are [?,18,18,?] and [?,18,8,1]. for 'dot_6/MatMul' (op: 'BatchMatMul') with input shapes: [?,18,18,?,18,18], [?,18,8,1,1,1].

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 8)         7544        user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 8)         13456       item_input[0][0]                 
__________________________________________________________________________________________________
flatten_6 

In [21]:
import numpy as np
a = tf.constant(np.zeros([2,5,4]), dtype=float)
b = tf.constant(np.zeros([2,5,4]), dtype=float)

<tf.Tensor 'Const:0' shape=(2, 5, 4) dtype=float32>

array([[[0., 0., 0.],
        [0., 0., 0.]]])